# Pytorch_8_Proste sieci neuronowe_autograd
https://github.com/jcjohnson/pytorch-examples#pytorch-custom-nn-modules
10:10  10:50
11:05  
Tutaj używamy PyTorch Tensors i autograd do wdrożenia naszej dwuwarstwowej sieci; teraz nie musimy już ręcznie wprowadzać wstecznego przejścia przez sieć:

In [1]:
import torch

Odpalam karte graficzną GPU

In [2]:
device = torch.device('cpu') # obliczenia robie na CPU
#device = torch.device('cuda') # obliczenia robie na GPU

### Dane
- N to wielkość partii; 
- D_in jest wymiarem wejściowym; 
- H jest ukrytym wymiarem; 
- D_out jest wymiarem wyjściowym. 


## <span style="color:red">Przyjmuje 10 zmiennych wynikowych! Coś nowego!</span>

In [3]:
N, D_in, H, D_out = 64, 1000, 100, 10

Powyżej zostały określone parametry aby tensor zmiennych niezależnych i tensor wynikowy były odpowiednie 

x: 64 obserwacji i 1000 zmiennych

y: 64 obserwacji i 10 zmiennych

### Tworzy losowe dane wejściowe i wyjściowe 

In [4]:
x = torch.randn(N, D_in, device=device)
y = torch.randn(N, D_out, device=device)

In [5]:
x.shape

torch.Size([64, 1000])

In [6]:
print('x ',x.size())
print('y ',y.size())

x  torch.Size([64, 1000])
y  torch.Size([64, 10])


### Tworzy wagi losowe dla x i y <span style="color:red">(requires_grad=True)</span>

Utwórzenie losowych tensorów dla wag; ustawienie 'requires_grad=True' oznacza, że my
chce obliczyć gradienty dla tych Tensorów podczas przebiegu wstecznego.

In [7]:
w1 = torch.randn(D_in, H, device=device, requires_grad=True)
w2 = torch.randn(H, D_out, device=device, requires_grad=True)

In [8]:
w1

tensor([[ 0.2565,  0.4659,  0.8209,  ...,  0.1362,  0.6051,  0.6771],
        [-0.9026,  2.0500, -0.1118,  ..., -0.3529, -0.0528,  1.0126],
        [ 0.2799, -0.2827,  0.4798,  ...,  0.3460,  2.0848,  0.0809],
        ...,
        [ 1.2322,  0.5567, -1.6159,  ...,  1.9356,  2.2632,  0.6829],
        [-2.0807, -0.2646,  0.9204,  ..., -2.0875, -1.9105,  2.3130],
        [-2.0963,  0.6964,  1.9831,  ..., -0.3588, -1.5690, -1.8868]],
       requires_grad=True)

In [9]:
print('w1 dla x: ', w1.shape)
print('w2 dla y: ', w2.shape)

w1 dla x:  torch.Size([1000, 100])
w2 dla y:  torch.Size([100, 10])


### Definiowanie nauki

Przekaz do przodu: oblicza przewidywane y przy użyciu operacji na Tensorach. Ponieważ w1 i w2 wymagają 'requires_grad=True' , operacje obejmujące te Tensory spowodują, że PyTorch zbuduje wykres obliczeniowy, umożliwiając automatyczne obliczanie gradientów. Ponieważ nie wdrażamy już ręcznie przekazywania wstecznego (backward pass), nie musimy przechowywać odniesień do wartości pośrednich.

In [10]:
learning_rate = 1e-6
for t in range(500):

   y_pred = x.mm(w1).clamp(min=0).mm(w2)
  
      # Funkcja błędu r2 dla drukowana dla każdej epoki
   loss = (y_pred - y).pow(2).sum()
   print('Epoka:',t, loss.item())
    
       #KROK 1
       # Użyj programu autograd, aby obliczyć przebieg wsteczny. To połączenie obliczy
       # gradient straty w stosunku do wszystkich Tensorów z 'requires_grad=True'.
       # Po tym wywołaniu w1.grad i w2.grad będą Tensorami trzymającymi gradient
       # straty w odniesieniu odpowiednio do w1 i w2.
   loss.backward()

       #KROK 2
       # Zaktualizuj wagi przy użyciu spadku gradientu. W tym kroku chcemy po prostu mutować
       # wartości w1 i w2 w miejscu; nie chcemy budować obliczeń
       # wykres dla kroków aktualizacji, dlatego używamy menedżera kontekstu torch.no_grad ()
       #, aby zapobiec tworzeniu przez PyTorch wykresu obliczeniowego dla aktualizacji
   with torch.no_grad():
    w1 -= learning_rate * w1.grad
    w2 -= learning_rate * w2.grad

       #KROK 3
       # Ręcznie zeruj gradienty po przejściu do tyłu
    w1.grad.zero_()
    w2.grad.zero_()
    
    

Epoka: 0 31434024.0
Epoka: 1 24057264.0
Epoka: 2 19737698.0
Epoka: 3 15852058.0
Epoka: 4 12030814.0
Epoka: 5 8558973.0
Epoka: 6 5852370.0
Epoka: 7 3944561.25
Epoka: 8 2699839.5
Epoka: 9 1907425.25
Epoka: 10 1403147.625
Epoka: 11 1074192.5
Epoka: 12 851468.875
Epoka: 13 693502.1875
Epoka: 14 576606.75
Epoka: 15 486875.21875
Epoka: 16 415898.03125
Epoka: 17 358408.84375
Epoka: 18 311070.09375
Epoka: 19 271532.40625
Epoka: 20 238139.90625
Epoka: 21 209665.59375
Epoka: 22 185242.75
Epoka: 23 164179.84375
Epoka: 24 145933.5625
Epoka: 25 130049.171875
Epoka: 26 116174.15625
Epoka: 27 104008.1015625
Epoka: 28 93336.734375
Epoka: 29 83940.3125
Epoka: 30 75634.484375
Epoka: 31 68284.5390625
Epoka: 32 61762.484375
Epoka: 33 55954.58203125
Epoka: 34 50771.3203125
Epoka: 35 46137.99609375
Epoka: 36 41988.0078125
Epoka: 37 38267.265625
Epoka: 38 34924.54296875
Epoka: 39 31914.67578125
Epoka: 40 29199.03515625
Epoka: 41 26745.41796875
Epoka: 42 24524.013671875
Epoka: 43 22511.451171875
Epoka: 44 206

Epoka: 292 0.07539603114128113
Epoka: 293 0.07238535583019257
Epoka: 294 0.06944560259580612
Epoka: 295 0.06669138371944427
Epoka: 296 0.06399346888065338
Epoka: 297 0.06142868101596832
Epoka: 298 0.05896827578544617
Epoka: 299 0.05660422891378403
Epoka: 300 0.0543401800096035
Epoka: 301 0.05216669291257858
Epoka: 302 0.05006982013583183
Epoka: 303 0.04807579889893532
Epoka: 304 0.04617215320467949
Epoka: 305 0.04431772977113724
Epoka: 306 0.04253913462162018
Epoka: 307 0.04086308926343918
Epoka: 308 0.03923500329256058
Epoka: 309 0.03766511380672455
Epoka: 310 0.036161474883556366
Epoka: 311 0.03473174571990967
Epoka: 312 0.033366914838552475
Epoka: 313 0.03203557804226875
Epoka: 314 0.030761081725358963
Epoka: 315 0.029532041400671005
Epoka: 316 0.028375394642353058
Epoka: 317 0.02725154347717762
Epoka: 318 0.0261791180819273
Epoka: 319 0.025143470615148544
Epoka: 320 0.024143878370523453
Epoka: 321 0.023195799440145493
Epoka: 322 0.02230057492852211
Epoka: 323 0.021406326442956924
E